In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

# Install on Google Colab
import subprocess
import sys

from IPython import get_ipython

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["ampform[doc]", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

# K-matrix

While {mod}`ampform` does not yet provide a generic way to formulate an amplitude model with $\boldsymbol{K}$-matrix dynamics, the (experimental) {mod}`.kmatrix` module makes it fairly simple to produce a symbolic expression for a parameterized $\boldsymbol{K}$-matrix with arbitrary resonances and channels and play around with it interactively. For more info on the $\boldsymbol{K}$-matrix, see the classic paper by Chung {cite}`chungPartialWaveAnalysis1995`, {pdg-review}`Resonances`, or this instructive presentation {cite}`meyerMatrixTutorial2008`.

Section {ref}`usage/dynamics/k-matrix:Physics` provides a summary of {cite}`chungPartialWaveAnalysis1995`, so that the equations can be referenced to in the {mod}`.kmatrix` module. It also points out some subtleties and deviations.

:::{note}

The $\boldsymbol{K}$-matrix approach was worked originally worked out in {doc}`compwa-org:report/005`, {doc}`compwa-org:report/009`, and {doc}`compwa-org:report/010`. Those reports contained a few mistakes, which have been (and are being) addressed here.

:::

In [ ]:
%matplotlib widget
import logging
import re
import warnings

import graphviz
import ipywidgets
import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt
import numpy as np
import sympy as sp
from IPython.display import Image
from matplotlib import cm
from mpl_interactions.controller import Controls

import symplot
from ampform import dynamics
from ampform.dynamics.kmatrix import TMatrix

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

warnings.filterwarnings("ignore")

## Physics

The $\boldsymbol{K}$-matrix formalism is used to describe coupled, two-body **formation processes** of the form $c_j d_j \to R \to a_i b_i$, with $i,j$ representing each separate channel and $R$ a number of resonances by which these channels are coupled.

In [ ]:
dot = """
digraph {
    rankdir=LR;
    node [shape=point, width=0];
    edge [arrowhead=none];
    "Na" [shape=none, label="aᵢ"];
    "Nb" [shape=none, label="bᵢ"];
    "Nc" [shape=none, label="cⱼ"];
    "Nd" [shape=none, label="dⱼ"];
    { rank=same "Nc", "Nd" };
    { rank=same "Na", "Nb" };
    "Nc" -> "N0";
    "Nd" -> "N0";
    "N1" -> "Na";
    "N1" -> "Nb";
    "N0" -> "N1" [label="R"];
    "N0" [shape=none, label=""];
    "N1" [shape=none, label=""];
}
"""
graphviz.Source(dot)

A small adaptation allows us to describe a coupled, two-body **production process** of the form $R \to a_ib_i$ (see {ref}`usage/dynamics/k-matrix:Production processes`).

In the following, $n$ denotes the number of channels and $n_R$ the number of resonances. In the {mod}`.kmatrix` module, we use $0 \leq i,j < n$ and $1 \leq R \leq n_R$.

### Partial wave expansion

In amplitude analysis, the main aim is to express the differential cross section $\frac{d\sigma}{d\Omega}$, that is, the intensity distribution in each spherical direction $\Omega=(\phi,\theta)$ as we can observe in experiments. This differential cross section can be expressed in terms of the **scattering amplitude** $A$:

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eq. (1)
```

$$
\frac{d\sigma}{d\Omega} = \left|A(\Omega)\right|^2.
$$ (differential cross section)

We can now further express $A$ in terms of **partial wave amplitudes** by splitting it up in terms of its angular momentum components:[^spin-formalisms]

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eq. (2)
```

$$
A(\Omega) = \frac{1}{q_i}\sum_L\left(2L+1\right) T_L(s) {D^{*L}_{\lambda\mu}}\left(\phi,\theta,0\right)
$$ (partial wave expansion)

with $L$ the total angular momentum of the decay product pair, $\lambda=\lambda_a-\lambda_b$ and $\mu=\lambda_c-\lambda_d$ the helicity differences of the final and initial states, $D$ a [Wigner-$D$ function](https://en.wikipedia.org/wiki/Wigner_D-matrix), and $T_J$ an operator representing the partial wave amplitude.

[^spin-formalisms]: Further subtleties arise when taking spin into account, especially for sequential decays. This is where {doc}`spin formalisms </usage/formalism>` come in.

The above sketch is just with one channel in mind. The same holds true though for a number of channels $n$, with the only difference being that the $T$ operator becomes a $\boldsymbol{T}$-matrix of rank $n$.

### Transition operator

The important point is that we have now expressed $A$ in terms of an angular part (depending on $\Omega$) and a dynamical part $\boldsymbol{T}$ that depends on the {ref}`Mandelstam variable <pwa:introduction:Mandelstam variables>` $s$.


The dynamical part $\boldsymbol{T}$ is usually called the **transition operator**. It describes the interacting part of the more general **scattering operator** $\boldsymbol{S}$, which describes the (complex) amplitude $\langle f|\boldsymbol{S}|i\rangle$  of an initial state $|i\rangle$ transitioning to a final state $|f\rangle$. The scattering operator describes both the non-interacting amplitude and the transition amplitude, so it relates to the transition operator as:

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eq. (10)
```

$$
\boldsymbol{S} = \boldsymbol{I} + 2i\boldsymbol{T}
$$ (S in terms of T)

with $\boldsymbol{I}$ the identity operator. Just like in {cite}`chungPartialWaveAnalysis1995`, we use a factor 2, while other authors choose $\boldsymbol{S} = \boldsymbol{I} + i\boldsymbol{T}$. In that case, one would have to multiply Eq. {eq}`partial wave expansion` by a factor $\frac{1}{2}$.

### Ensuring unitarity

Knowing the origin of the $\boldsymbol{T}$-matrix, there is an important restriction that we need to comply with when we further formulate a {ref}`parametrization <usage/dynamics/k-matrix:Resonance parametrization>`: **unitarity**. This means that $\boldsymbol{S}$ should conserve probability, namely $\boldsymbol{S}^\dagger\boldsymbol{S} = \boldsymbol{I}$. Luckily, there is a trick that makes this easier. If we express $\boldsymbol{S}$ in terms of an operator $\boldsymbol{K}$ by applying a [Cayley transformation](https://en.wikipedia.org/wiki/Cayley_transform):

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eq. (20)
```

$$
\boldsymbol{S} = (\boldsymbol{I} + i\boldsymbol{K})(I - i\boldsymbol{K})^{-1},
$$ (Cayley transformation)

_unitarity is conserved if $\boldsymbol{K}$ is real_. With some matrix jumbling, we can derive that the $\boldsymbol{T}$-matrix can be expressed in terms of $\boldsymbol{K}$ as follows:

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eq. (19);
compare with {eq}`T-hat in terms of K-hat`
```

$$
\boldsymbol{T}
= \boldsymbol{K} \left(\boldsymbol{I} - i\boldsymbol{K}\right)^{-1}
= \left(\boldsymbol{I} - i\boldsymbol{K}\right)^{-1} \boldsymbol{K}.
$$ (T in terms of K)

### Lorentz-invariance

The description so far did not take Lorentz-invariance into account. For this, we first need to define a **two-body phase space matrix** $\boldsymbol{\rho}$:

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eq. (36)
```

$$
\boldsymbol{\rho} = \begin{pmatrix}
\rho_0 & \cdots & 0      \\
\vdots & \ddots & \vdots \\
0      & \cdots & \rho_{n-1}
\end{pmatrix}.
$$ (rho matrix)

The **Lorentz-invariant amplitude $\boldsymbol{\hat{T}}$** and corresponding Lorentz-invariant $\boldsymbol{\hat{K}}$-matrix can then be computed from $\boldsymbol{T}$ and $\boldsymbol{K}$ with:[^rho-dagger]

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eqs. (34) and (47)
```

$$
\begin{eqnarray}
\boldsymbol{T} & = & \sqrt{\boldsymbol{\rho}} \; \boldsymbol{\hat{T}} \sqrt{\boldsymbol{\rho}} \\
\boldsymbol{K} & = & \sqrt{\boldsymbol{\rho}} \; \boldsymbol{\hat{K}} \sqrt{\boldsymbol{\rho}}.
\end{eqnarray}
$$ (K-hat and T-hat)

[^rho-dagger]: An unpublished primer on the $\boldsymbol{K}$-matrix by Chung {cite}`chungPrimerKmatrixFormalism1995` uses a conjugate transpose of $\boldsymbol{\rho}$, e.g. $\boldsymbol{T} = \sqrt{\boldsymbol{\rho^\dagger}} \; \boldsymbol{\hat{T}} \sqrt{\boldsymbol{\rho}}$. This should not matter above threshold, where the phase space factor is real, but could have effects below threshold. This is where things become tricky: on the one hand, we need to ensure that $\boldsymbol{K}$ remains real (unitarity) and on the other, we need to take keep track of which imaginary square root we choose (**Riemann sheet**). The latter is often called the requirement of **analyticity**. This is currently being explored in {doc}`compwa-org:report/003` and {doc}`compwa-org:report/004`.

With these definitions, we can deduce that:

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eq. (51);
compare with {eq}`T in terms of K`
```

$$
\boldsymbol{\hat{T}}
= \boldsymbol{\hat{K}} (\boldsymbol{I} - i\boldsymbol{\rho}\boldsymbol{\hat{K}})^{-1}
= (\boldsymbol{I} - i\boldsymbol{\hat{K}}\boldsymbol{\rho})^{-1} \boldsymbol{\hat{K}}.
$$ (T-hat in terms of K-hat)

### Production processes

{ref}`As noted in the intro <usage/dynamics/k-matrix:Physics>`, the $\boldsymbol{K}$-matrix describes scattering processes of type $cd \to ab$. It can however be generalized to describe **production processes** of type $R \to ab$. Here, the amplitude is described by a **final state $F$-vector** of size $n$, so the question is how to express $F$ in terms of transition matrix $\boldsymbol{T}$.

In [ ]:
dot = """
digraph {
    rankdir=LR;
    node [shape=point, width=0];
    edge [arrowhead=none];
    "a" [shape=none, label="aᵢ"];
    "b" [shape=none, label="bᵢ"];
    "R" [shape=none, label="R"];
    "N" [shape=none, label=""];
    "R" -> "N";
    "N" -> "a";
    "N" -> "b";
    { rank=same "Na", "Nb" };
}
"""
graphviz.Source(dot)

One approach by {cite}`aitchisonMatrixFormalismOverlapping1972` is to transform $\boldsymbol{T}$ into $F$ (and its relativistic form $\hat{F}$) through the **production amplitude $P$-vector**:

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eqs. (114) and (115)
```

$$
\begin{eqnarray}
F & = & \left(\boldsymbol{I}-i\boldsymbol{K}\right)^{-1}P \\
\hat{F} & = & \left(\boldsymbol{I}-i\boldsymbol{\hat{K}\boldsymbol{\rho}}\right)^{-1}\hat{P},
\end{eqnarray}
$$ (F in terms of P)

where we can compute $\boldsymbol{\hat{K}}$ from {eq}`K-hat and T-hat`:

$$
\hat{\boldsymbol{K}} = \sqrt{\boldsymbol{\rho}^{-1}} \boldsymbol{K} \sqrt{\boldsymbol{\rho}^{-1}}.
$$ (K-hat in terms of K)

Another approach by {cite}`cahnMystery9801986` further approximates this by defining a **$Q$-vector**:

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eq. (124)
```

$$
Q = \boldsymbol{K}^{-1}P \quad \mathrm{and} \quad
\hat{Q} = \boldsymbol{\hat{K}}^{-1}\hat{P}
$$ (Q-vector)

that _is taken to be constant_ (just some 'fitting' parameters). The $F$-vector can then be expressed as:

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eq. (125)
```

$$
F = \boldsymbol{T}Q
\quad \mathrm{and} \quad
\hat{F} = \boldsymbol{\hat{T}}\hat{Q}
$$ (F in terms of Q)

Note that for all these vectors, we have:

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eqs. (116) and (124)
```

$$
F=\sqrt{\boldsymbol{\rho}}\hat{F},\quad
P=\sqrt{\boldsymbol{\rho}}\hat{P},\quad\mathrm{and}\quad
Q=\sqrt{\boldsymbol{\rho}^{-1}}\hat{Q}.
$$ (invariant vectors)

### Resonance parametrization

After all these matrix definitions, the final challenge is to choose a correct parametrization for the elements of $\boldsymbol{K}$ and $P$ that accurately describes the resonances we observe. There are several choices, but a common one is the following summation over the **poles** $R$:[^complex-conjugate-parametrization]

[^complex-conjugate-parametrization]: Eqs. (51) and (52) in {cite}`chungPrimerKmatrixFormalism1995` take a complex conjugate of one of the residue functions and one of the phase space factors.

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eqs. (73) and (74)
```

$$
\begin{eqnarray}
K_{ij} &=& \sum_R\frac{g_{R,i}g_{R,j}}{m_R^2-s} + c_{ij} \\
\hat{K}_{ij} &=& \sum_R \frac{g_{R,i}(s)g_{R,j}(s)}{\left(m_R^2-s\right)\sqrt{\rho_i\rho_j}} + \hat{c}_{ij}
\end{eqnarray}
$$ (K-matrix parametrization)

with $c_{ij}, \hat{c}_{ij}$ some optional background characterization and $g_{R,i}$ the **residue functions**. The residue functions are often further expressed as:

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eqs. (75-78)
```

$$
\begin{eqnarray}
g_{R,i} &=& \gamma_{R,i}\sqrt{m_R\Gamma^0_{R,i}} \\
g_{R,i}(s) &=& \gamma_{R,i}\sqrt{m_R\Gamma_{R,i}(s)}
\end{eqnarray}
$$ (residue function)

with $\gamma_{R,i}$ some _real_ constants and $\Gamma^0_{R,i}$ the **partial width** of each pole. In the Lorentz-invariant form, the fixed width $\Gamma^0$ is replaced by an "energy dependent" {class}`.CoupledWidth` $\Gamma(s)$.[^phase-space-factor-normalization] The **width** for each pole can be computed as $\Gamma^0_R = \sum_i\Gamma^0_{R,i}$.

[^phase-space-factor-normalization]: Unlike Eq. (77) in {cite}`chungPartialWaveAnalysis1995`, AmpForm defines {class}`.CoupledWidth` as in {pdg-review}`2020; Resonances; p.6`, Eq. (49.21). The difference is that the phase space factor denoted by $\rho_i$ in Eq. (77) in {cite}`chungPartialWaveAnalysis1995` is divided by the phase space factor of the resonance mass. So in AmpForm, the choice is $\rho_i \to \frac{\rho_i(s)}{\rho_R(s)}$.

The production vector $P$ is commonly parameterized as:[^damping-factor-P-parametrization]

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eqs. (118-119) and (122)
```

$$
\begin{eqnarray}
P_i &=& \sum_R \frac{\beta^0_R\,g_{R,i}(s)}{m_R^2-s} \\
\hat{P}_i
&=& \sum_R \frac{\beta^0_R\,g_{R,i}(s)}{\left(m_R^2-s\right)\sqrt{\rho_i}} \\
&=& \sum_R \frac{
    \beta^0_R\gamma_{R,i}m_R\Gamma^0_R B_{R,i}(q(s))
}{m_R^2-s}
\end{eqnarray}
$$ (P-vector parametrization)

with $B_{R,i}(q(s))$ the **centrifugal damping factor** (see {class}`~ampform.dynamics.BlattWeisskopfSquared`) for channel $i$ and $\beta_R^0$ some (generally complex) constants that describe the production information of resonance $R$. Usually, these constants are rescaled just like the residue functions in {eq}`residue function`:

[^damping-factor-P-parametrization]: Just as with [^phase-space-factor-normalization], we have smuggled a bit in the last equation in order to be able to reproduce Eq. (49.19) in {pdg-review}`2020; Resonances; p.6` in the case $n=1,n_R=1$, on which {func}`.relativistic_breit_wigner_with_ff` is based.

```{margin}
{cite}`chungPartialWaveAnalysis1995` Eq. (121)
```

$$
\beta^0_R = \beta_R\sqrt{m_R\Gamma^0_R}.
$$ (beta functions)

## Implementation

### Non-relativistic K-matrix

A non-relativistic $\boldsymbol{K}$-matrix for an arbitrary number of channels and an arbitrary number of resonances can be formulated with the {meth}`.NonRelativisticKMatrix.formulate` method:

In [ ]:
from ampform.dynamics.kmatrix import NonRelativisticKMatrix

n_resonances = sp.Symbol("n_R", integer=True, positive=True)
k_matrix_nr = NonRelativisticKMatrix.formulate(
    n_resonances=n_resonances, n_channels=1
)
k_matrix_nr[0, 0]

Notice how the $\boldsymbol{K}$-matrix reduces to a {func}`.relativistic_breit_wigner` when there is one channel and one resonance (but for a residue constant $\gamma$):

In [ ]:
k_matrix_1r = NonRelativisticKMatrix.formulate(n_resonances=1, n_channels=1)
k_matrix_1r[0, 0].doit().simplify()

Now let's investigate the effect of using a $\boldsymbol{K}$-matrix to describe **two resonances** in one channel and see how it compares with the sum of two Breit-Wigner functions. Two Breit-Wigner 'poles' with the same parameters would look like this:

In [ ]:
from ampform.dynamics import relativistic_breit_wigner

s, m1, m2, Gamma1, Gamma2 = sp.symbols("s, m1, m2, Gamma1, Gamma2")
bw1 = relativistic_breit_wigner(s, m1, Gamma1)
bw2 = relativistic_breit_wigner(s, m2, Gamma2)
bw = bw1 + bw2
bw

while a $\boldsymbol{K}$-matrix parametrizes the two resonances as:

In [ ]:
k_matrix_2r = NonRelativisticKMatrix.formulate(n_resonances=2, n_channels=1)
k_matrix = k_matrix_2r[0, 0].doit()

In [ ]:
# reformulate terms
denominator, nominator = k_matrix.args
term1 = nominator.args[0] * denominator
term2 = nominator.args[1] * denominator
k_matrix = term1 + term2
k_matrix

To simplify things, we can set the residue constants $\gamma$ to one. Notice how the $\boldsymbol{K}$-matrix has introduced some coupling ('interference') between the two terms.

In [ ]:
def remove_residue_constants(expression):
    expression = symplot.substitute_indexed_symbols(expression)
    residue_constants = filter(
        lambda s: re.match(r"^\\?gamma", s.name),
        expression.free_symbols,
    )
    return expression.xreplace({gamma: 1 for gamma in residue_constants})


display(
    remove_residue_constants(bw),
    remove_residue_constants(k_matrix),
)

Now, just like in {doc}`/usage/interactive`, we use {mod}`symplot` to visualize the difference between the two expressions. The important thing is that the Argand plot on the right shows that **the $\boldsymbol{K}$-matrix conserves unitarity**.

Note that we have to call {func}`symplot.substitute_indexed_symbols` to turn the {class}`~sympy.tensor.indexed.Indexed` instances in this {obj}`~sympy.matrices.dense.Matrix` expression into {class}`~sympy.core.symbol.Symbol`s before calling this function. We also call {func}`symplot.rename_symbols` so that the residue $\gamma$'s get a name that does not have to be dummified by {func}`~sympy.utilities.lambdify.lambdify`.

In [ ]:
# Prepare expressions
m = sp.Symbol("m")
kmatrix = symplot.substitute_indexed_symbols(k_matrix)
rename_gamma = lambda s: re.sub(  # noqa: E731
    r"\\gamma_{([0-9]),0}", r"gamma\1", s
)
gamma1, gamma2 = sp.symbols("gamma1 gamma2")
bw = symplot.rename_symbols(bw, rename_gamma)
kmatrix = symplot.rename_symbols(kmatrix, rename_gamma)
bw = bw.xreplace({s: m ** 2})
kmatrix = kmatrix.xreplace({s: m ** 2})

# Prepare sliders and domain
np_kmatrix, sliders = symplot.prepare_sliders(kmatrix, m)
np_bw = sp.lambdify((m, Gamma1, Gamma2, gamma1, gamma2, m1, m2), bw.doit())

m_min, m_max = 0, 3
domain_1d = np.linspace(m_min, m_max, 200)
domain_argand = np.linspace(m_min - 2, m_max + 2, 1_000)
sliders.set_ranges(
    m1=(0, 3, 100),
    m2=(0, 3, 100),
    Gamma1=(0, 2, 100),
    Gamma2=(0, 2, 100),
    gamma1=(0, 2),
    gamma2=(0, 2),
)
sliders.set_values(
    m1=1.1,
    m2=1.9,
    Gamma1=0.2,
    Gamma2=0.3,
    gamma1=1,
    gamma2=1,
)
if STATIC_WEB_PAGE:
    # Concatenate flipped domain for reverse animation
    domain = np.linspace(1.0, 2.7, 50)
    domain = np.concatenate((domain, np.flip(domain[1:])))
    sliders._sliders["m1"] = domain


def create_argand(func):
    def wrapped(**kwargs):
        values = func(domain_argand, **kwargs)
        argand = np.array([values.real, values.imag])
        return argand.T

    return wrapped


# Create figure
fig, axes = plt.subplots(
    ncols=2,
    figsize=1.2 * np.array((8, 3.8)),
    tight_layout=True,
)
ax_intensity, ax_argand = axes
m_label = "$m_{a+b}$"
ax_intensity.set_xlabel(m_label)
ax_intensity.set_ylabel("$|A|^2$")
ax_argand.set_xlabel("Re($A$)")
ax_argand.set_ylabel("Im($A$)")

# Plot intensity
controls = iplt.plot(
    domain_1d,
    lambda *args, **kwargs: np.abs(np_kmatrix(*args, **kwargs) ** 2),
    label="$K$-matrix",
    **sliders,
    ylim="auto",
    ax=ax_intensity,
)
iplt.plot(
    domain_1d,
    lambda *args, **kwargs: np.abs(np_bw(*args, **kwargs) ** 2),
    label="Breit-Wigner",
    controls=controls,
    ylim="auto",
    ax=ax_intensity,
)
plt.legend(loc="upper right")
iplt.axvline(controls["m1"], c="gray", linestyle="dotted")
iplt.axvline(controls["m2"], c="gray", linestyle="dotted")

# Argand plots
iplt.scatter(
    create_argand(np_kmatrix),
    label="$K$-matrix",
    controls=controls,
    parametric=True,
    s=1,
    ax=ax_argand,
)
iplt.scatter(
    create_argand(np_bw),
    label="Breit-Wigner",
    controls=controls,
    parametric=True,
    s=1,
    ax=ax_argand,
)
plt.legend(loc="upper right");

{{ run_interactive }}

In [ ]:
if STATIC_WEB_PAGE:
    output_path = "non-relativistic-k-matrix.gif"
    ax_intensity.set_ylim([0, 2])
    ax_argand.set_xlim([-1, +1])
    ax_argand.set_ylim([0, 2])
    controls.save_animation(output_path, fig, "m1", fps=20)
    with open(output_path, "rb") as f:
        display(Image(data=f.read(), format="png"))

### Relativistic K-matrix

Relativistic $\boldsymbol{K}$-matrices for an arbitrary number of channels and an arbitrary number of resonances can be formulated with the {meth}`.RelativisticKMatrix.formulate` method:

In [ ]:
from ampform.dynamics.kmatrix import RelativisticKMatrix

n_resonances = sp.Symbol("n_R", integer=True, positive=True)
rel_k_matrix_nr = RelativisticKMatrix.formulate(
    n_resonances=n_resonances, n_channels=1
)
rel_k_matrix_nr[0, 0]

Again, as in {ref}`usage/dynamics/k-matrix:Non-relativistic K-matrix`, the $\boldsymbol{K}$-matrix reduces to something of a {func}`.relativistic_breit_wigner`. This time, the width has been replaced by a {class}`.CoupledWidth` and some {class}`.PhaseSpaceFactor`s have been inserted that take case of the decay into two decay products:[^conjugate-phsp-factor]

[^conjugate-phsp-factor]: The two {class}`.PhaseSpaceFactor`s $\sqrt{\rho_0(s)}$ cancel out within phase space, where they are real.

In [ ]:
rel_k_matrix_1r = RelativisticKMatrix.formulate(n_resonances=1, n_channels=1)
symplot.partial_doit(rel_k_matrix_1r[0, 0], sp.Sum).simplify(doit=False)

Similarly, the $\boldsymbol{K}$-matrix with two resonances becomes (neglecting the $\sqrt{\rho_0(s)}$):

In [ ]:
rel_k_matrix_2r = RelativisticKMatrix.formulate(n_resonances=2, n_channels=1)
rel_k_matrix_2r = symplot.partial_doit(rel_k_matrix_2r[0, 0], sp.Sum)

In [ ]:
rel_k_matrix_2r = symplot.substitute_indexed_symbols(rel_k_matrix_2r)
s, m_a, m_b = sp.symbols("s, m_a0, m_b0")
rho = dynamics.PhaseSpaceFactor(s, m_a, m_b)
rel_k_matrix_2r = rel_k_matrix_2r.xreplace(
    {
        sp.sqrt(rho): 1,
        sp.conjugate(sp.sqrt(rho)): 1,
    }
)
denominator, nominator = rel_k_matrix_2r.args
term1 = nominator.args[0] * denominator
term2 = nominator.args[1] * denominator
rel_k_matrix_2r = term1 + term2
rel_k_matrix_2r

### P-vector

For one channel and an arbitrary number of resonances $n_R$, the $F$-vector gets the following form:

In [ ]:
from ampform.dynamics.kmatrix import NonRelativisticPVector

n_resonances = sp.Symbol("n_R", integer=True, positive=True)
f_vector_nr = NonRelativisticPVector.formulate(
    n_resonances=n_resonances, n_channels=1
)
f_vector_nr[0]

As in {ref}`usage/dynamics/k-matrix:Non-relativistic K-matrix`, if we take $n_R=1$, the $F$-vector reduces to a Breit-Wigner function.

In [ ]:
f_vector_1r = NonRelativisticPVector.formulate(n_resonances=1, n_channels=1)
f_vector_1r[0].doit().simplify()

Note that the $F$-vector approach introduces two additional coefficients $\beta$. These can constants can be complex and can introduce phase differences.

In [ ]:
f_vector_2r = NonRelativisticPVector.formulate(n_resonances=2, n_channels=1)
f_vector = f_vector_2r[0].doit()

In [ ]:
denominator, nominator = f_vector.args
term1 = nominator.args[0] * denominator
term2 = nominator.args[1] * denominator
f_vector = term1 + term2
f_vector

Now again let's compare the compare this with a sum of two {func}`.relativistic_breit_wigner`s, now with the two additional $\beta$-constants.

In [ ]:
beta1, beta2 = sp.symbols("beta1 beta2")
bw_with_phases = beta1 * bw1 + beta2 * bw2
display(
    bw_with_phases,
    remove_residue_constants(f_vector),
)

{{ run_interactive }}

In [ ]:
# Prepare expressions
f_vector = symplot.substitute_indexed_symbols(f_vector)
rename_gamma = lambda s: re.sub(  # noqa: E731
    r"\\gamma_{([0-9]),0}", r"gamma\1", s
)
c1, c2, phi1, phi2 = sp.symbols("c1 c2 phi1 phi2", real=True)
bw_with_phases = symplot.rename_symbols(bw_with_phases, rename_gamma)
f_vector = symplot.rename_symbols(f_vector, rename_gamma)
substitutions = {
    s: m ** 2,
    beta1: c1 * sp.exp(sp.I * phi1),
    beta2: c2 * sp.exp(sp.I * phi2),
}
bw_with_phases = bw_with_phases.xreplace(substitutions)
f_vector = f_vector.xreplace(substitutions)

# Prepare sliders and domain
np_f_vector, sliders = symplot.prepare_sliders(f_vector, m)
np_bw = sp.lambdify(
    (m, Gamma1, Gamma2, c1, c2, gamma1, gamma2, m1, m2, phi1, phi2),
    bw_with_phases.doit(),
)

# Set plot domain
x_min, x_max = 0, 3
y_min, y_max = -0.5, +0.5
plot_domain = np.linspace(x_min, x_max, num=150)
plot_domain_argand = np.linspace(x_min - 2, x_max + 2, num=400)
x_values = np.linspace(x_min, x_max, num=160)
y_values = np.linspace(y_min, y_max, num=80)
X, Y = np.meshgrid(x_values, y_values)
plot_domain_complex = X + Y * 1j

# Set slider values and ranges
sliders.set_ranges(
    c1=(0, 2, 100),
    c2=(0, 2, 100),
    phi1=(0, np.pi, np.pi / 20),
    phi2=(0, np.pi, np.pi / 20),
    m1=(0, 3, 100),
    m2=(0, 3, 100),
    Gamma1=(-2, 2, 100),
    Gamma2=(-2, 2, 100),
    gamma1=(0, 2),
    gamma2=(0, 2),
)
sliders.set_values(
    c1=1,
    c2=1,
    m1=1.4,
    m2=1.7,
    Gamma1=0.2,
    Gamma2=0.3,
    gamma1=1,
    gamma2=1,
)


def create_argand(func):
    def wrapped(**kwargs):
        values = func(plot_domain_argand, **kwargs)
        argand = np.array([values.real, values.imag])
        return argand.T

    return wrapped


# Create figure
fig, axes = plt.subplots(
    nrows=3,
    ncols=2,
    figsize=(10, 9),
    gridspec_kw=dict(
        width_ratios=[2.5, 1],
        wspace=0.08,
        left=0.05,
        right=0.99,
        top=0.99,
        bottom=0.05,
    ),
)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
(ax_1d, ax_argand), (ax_2d, ax_empty1), (ax_2d_bw, ax_empty2) = axes
ax_empty1.axis("off")
ax_empty2.axis("off")
for ax in axes.flatten():
    ax.set_yticks([])
ax_argand.set_xticks([])
ax_1d.axes.get_xaxis().set_visible(False)
ax_2d.axes.get_xaxis().set_visible(False)
ax_1d.sharex(ax_2d_bw)
ax_2d.sharex(ax_2d_bw)
ax_2d_bw.set_xlabel("Re $m$")
for ax in (ax_2d, ax_2d_bw):
    ax.set_ylabel("Im $m$")
ax_1d.set_ylabel("$|A|^2$")
ax_argand.set_xlabel("Re($A$)")
ax_argand.set_ylabel("Im($A$)")

for ax in (ax_2d, ax_2d_bw):
    ax.axhline(0, linewidth=0.5, c="black", linestyle="dotted")

# 1D intensity plot
controls = Controls(**sliders)
controls = iplt.plot(
    plot_domain,
    lambda *args, **kwargs: np.abs(np_f_vector(*args, **kwargs) ** 2),
    label="$P$-vector",
    controls=controls,
    ylim="auto",
    ax=ax_1d,
)
iplt.plot(
    plot_domain,
    lambda *args, **kwargs: np.abs(np_bw(*args, **kwargs) ** 2),
    label="Breit-Wigner",
    controls=controls,
    ylim="auto",
    ax=ax_1d,
)
ax_1d.legend(loc="upper right")
mass_line_style = dict(
    c="red",
    alpha=0.3,
)
for name in controls.params:
    if not re.match(r"^m[0-9]+$", name):
        continue
    iplt.axvline(controls[name], ax=ax_1d, **mass_line_style)

# Argand plot
iplt.scatter(
    create_argand(np_f_vector),
    label="$P$-vector",
    controls=controls,
    parametric=True,
    s=1,
    ax=ax_argand,
    xlim="auto",
    ylim="auto",
)
iplt.scatter(
    create_argand(np_bw),
    label="Breit-Wigner",
    controls=controls,
    parametric=True,
    s=1,
    ax=ax_argand,
    xlim="auto",
    ylim="auto",
)

# 3D plot
color_mesh = None
color_mesh_bw = None
resonances_indicators = []
resonances_indicators_bw = []


def plot3(*, z_cutoff, complex_rendering, **kwargs):
    global color_mesh, color_mesh_bw
    Z = np_f_vector(plot_domain_complex, **kwargs)
    Z_bw = np_bw(plot_domain_complex, **kwargs)
    if complex_rendering == "imag":
        projection = np.imag
        ax_title = "Re $A$"
    elif complex_rendering == "real":
        projection = np.real
        ax_title = "Im $A$"
    elif complex_rendering == "abs":
        projection = np.vectorize(lambda Z: np.abs(Z) ** 2)
        ax_title = "$|A|$"
    else:
        raise NotImplementedError
    ax_2d.set_title(ax_title + " ($P$-vector)")
    ax_2d_bw.set_title(ax_title + " (Breit-Wigner)")

    if color_mesh is None:
        color_mesh = ax_2d.pcolormesh(X, Y, projection(Z), cmap=cm.coolwarm)
    else:
        color_mesh.set_array(projection(Z)[:-1, :-1])
    color_mesh.set_clim(vmin=-z_cutoff, vmax=+z_cutoff)

    if color_mesh_bw is None:
        color_mesh_bw = ax_2d_bw.pcolormesh(
            X, Y, projection(Z_bw), cmap=cm.coolwarm
        )
    else:
        color_mesh_bw.set_array(projection(Z_bw)[:-1, :-1])
    color_mesh_bw.set_clim(vmin=-z_cutoff, vmax=+z_cutoff)

    if resonances_indicators:
        for R, (line, text) in enumerate(resonances_indicators, 1):
            mass = kwargs[f"m{R}"]
            line.set_xdata(mass)
            text.set_x(mass + (x_max - x_min) * 0.008)
    else:
        for R in range(1, 2 + 1):
            mass = kwargs[f"m{R}"]
            line = ax_2d.axvline(mass, **mass_line_style)
            text = ax_2d.text(
                x=mass + (x_max - x_min) * 0.008,
                y=0.95 * y_min,
                s=f"$m_{R}$",
                c="red",
            )
            resonances_indicators.append((line, text))

    if resonances_indicators_bw:
        for R, (line, text) in enumerate(resonances_indicators_bw, 1):
            mass = kwargs[f"m{R}"]
            line.set_xdata(mass)
            text.set_x(mass + (x_max - x_min) * 0.008)
    else:
        for R in range(1, 2 + 1):
            mass = kwargs[f"m{R}"]
            line = ax_2d_bw.axvline(mass, **mass_line_style)
            text = ax_2d_bw.text(
                x=mass + (x_max - x_min) * 0.008,
                y=0.95 * y_min,
                s=f"$m_{R}$",
                c="red",
            )
            resonances_indicators_bw.append((line, text))


# Create switch for imag/real/abs
name = "complex_rendering"
sliders._sliders[name] = ipywidgets.RadioButtons(
    options=["imag", "real", "abs"],
    description=R"\(s\)-plane plot",
)
sliders._arg_to_symbol[name] = name

# Create cut-off slider for z-direction
name = "z_cutoff"
sliders._sliders[name] = ipywidgets.FloatSlider(
    value=2,
    min=0,
    max=+5,
    step=0.1,
    description=R"\(z\)-cutoff",
)
sliders._arg_to_symbol[name] = name

# Create GUI
sliders_copy = dict(sliders)
slider_groups = []
for R in range(1, 2 + 1):
    vertical_slider_group = [
        sliders_copy.pop(f"c{R}"),
        sliders_copy.pop(f"phi{R}"),
        sliders_copy.pop(f"m{R}"),
        sliders_copy.pop(f"Gamma{R}"),
        sliders_copy.pop(f"gamma{R}"),
    ]
    slider_groups.append(vertical_slider_group)
slider_pairs = np.array(slider_groups).T
h_boxes = [ipywidgets.HBox(tuple(pair)) for pair in slider_pairs]
remaining_sliders = sorted(
    sliders_copy.values(),
    key=lambda s: (str(type(s)), s.description),
)
ui = ipywidgets.VBox(h_boxes + remaining_sliders)
output = ipywidgets.interactive_output(plot3, controls=sliders)
display(ui, output)

In [ ]:
if STATIC_WEB_PAGE:
    output_path = "p-vector-comparison.png"
    plt.savefig(output_path, dpi=150)
    display(Image(output_path))

## Interactive visualization

All $\boldsymbol{K}$-matrices can be inspected interactively for arbitrary resonances and channels with the following applet:

In [ ]:
L = sp.Symbol("L", integer=True, negative=False)
if STATIC_WEB_PAGE:
    L = 0


def plot(
    kmatrix_type: TMatrix,
    n_channels: int,
    n_resonances: int,
    angular_momentum: sp.Symbol = 0,
    phsp_factor=dynamics.PhaseSpaceFactor,
) -> None:
    # Convert to Symbol: symplot cannot handle IndexedBase
    i, j = sp.symbols("i, j", integer=True, negative=False)
    j = i
    expr = kmatrix_type.formulate(
        n_resonances=n_resonances,
        n_channels=n_channels,
        angular_momentum=angular_momentum,
        phsp_factor=phsp_factor,
    ).doit()[i, j]
    expr = expr.xreplace({s: m ** 2})
    expr = symplot.substitute_indexed_symbols(expr)
    np_expr, sliders = symplot.prepare_sliders(expr, m)

    # Set plot domain
    x_min, x_max = 1e-3, 3
    y_min, y_max = -0.5, +0.5

    plot_domain = np.linspace(x_min, x_max, num=500, dtype=np.complex)
    x_values = np.linspace(x_min, x_max, num=160)
    y_values = np.linspace(y_min, y_max, num=80)
    X, Y = np.meshgrid(x_values, y_values)
    plot_domain_complex = X + Y * 1j

    # Set slider values and ranges
    m0_values = np.linspace(x_min, x_max, num=n_resonances + 2)
    m0_values = m0_values[1:-1]
    if "L" in sliders:
        sliders.set_ranges(L=(0, 8))
    sliders.set_ranges(i=(0, n_channels - 1))
    for R in range(1, n_resonances + 1):
        for i in range(n_channels):
            if kmatrix_type is RelativisticKMatrix:
                sliders.set_ranges(
                    {
                        f"m{R}": (0, 3, 100),
                        fR"\Gamma_{{{R},{i}}}": (-2, +2, 100),
                        fR"\gamma_{{{R},{i}}}": (0, 10, 100),
                        f"m_a{i}": (0, 1, 0.01),
                        f"m_b{i}": (0, 1, 0.01),
                    }
                )
                sliders.set_values(
                    {
                        f"m{R}": m0_values[R - 1],
                        fR"\Gamma_{{{R},{i}}}": 2.0
                        * (0.4 + R * 0.2 - i * 0.3),
                        fR"\gamma_{{{R},{i}}}": 0.25 * (10 - R + i),
                        f"m_a{i}": (i + 1) * 0.25,
                        f"m_b{i}": (i + 1) * 0.25,
                    }
                )
            if kmatrix_type in {
                NonRelativisticPVector,
                NonRelativisticKMatrix,
            }:
                sliders.set_ranges(
                    {
                        f"m{R}": (0, 3, 100),
                        f"Gamma{R}": (-1, 1, 100),
                        fR"\gamma_{{{R},{i}}}": (0, 2, 100),
                    }
                )
                sliders.set_values(
                    {
                        f"m{R}": m0_values[R - 1],
                        f"Gamma{R}": (R + 1) * 0.1,
                        fR"\gamma_{{{R},{i}}}": 1 - 0.1 * R + 0.1 * i,
                    }
                )
            if kmatrix_type is NonRelativisticPVector:
                sliders.set_ranges({f"beta{R}": (-1, 1, 100)})
                sliders.set_values({f"beta{R}": 1})

    # Create interactive plots
    fig, axes = plt.subplots(
        nrows=2,
        figsize=(8, 6),
        sharex=True,
        tight_layout=True,
    )
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    if kmatrix_type in {NonRelativisticKMatrix, RelativisticKMatrix}:
        fig.suptitle(
            fR"${n_channels} \times {n_channels}$ $K$-matrix"
            f" with {n_resonances} resonances"
        )
    elif kmatrix_type is NonRelativisticPVector:
        fig.suptitle(
            f"$P$-vector for {n_channels} channels and {n_resonances} resonances"
        )

    for ax in axes:
        ax.set_xlim(x_min, x_max)
    ax_2d, ax_3d = axes
    ax_2d.set_ylabel("$|T|^{2}$")
    ax_2d.set_yticks([])
    ax_3d.set_xlabel("Re $m$")
    ax_3d.set_ylabel("Im $m$")
    ax_3d.set_xticks([])
    ax_3d.set_yticks([])

    ax_3d.axhline(0, linewidth=0.5, c="black", linestyle="dotted")

    # 2D plot
    def plot(channel: int):
        def wrapped(*args, **kwargs) -> sp.Expr:
            kwargs["i"] = channel
            return np.abs(np_expr(*args, **kwargs)) ** 2

        return wrapped

    controls = Controls(**sliders)
    for i in range(n_channels):
        iplt.plot(
            plot_domain,
            plot(i),
            ax=axes[0],
            controls=controls,
            ylim="auto",
            label=f"channel {i}",
        )
    if n_channels > 1:
        axes[0].legend(loc="upper right")
    mass_line_style = dict(
        c="red",
        alpha=0.3,
    )
    for name in controls.params:
        if not re.match(r"^m[0-9]+$", name):
            continue
        iplt.axvline(controls[name], ax=axes[0], **mass_line_style)

    # 3D plot
    color_mesh = None
    resonances_indicators = []
    threshold_indicators = []

    def plot3(*, z_cutoff, complex_rendering, **kwargs):
        nonlocal color_mesh
        Z = np_expr(plot_domain_complex, **kwargs)
        if complex_rendering == "imag":
            Z_values = Z.imag
            ax_title = "Re $T$"
        elif complex_rendering == "real":
            Z_values = Z.real
            ax_title = "Im $T$"
        elif complex_rendering == "abs":
            Z_values = np.abs(Z)
            ax_title = "$|T|$"
        else:
            raise NotImplementedError

        if n_channels == 1:
            axes[-1].set_title(ax_title)
        else:
            i = kwargs["i"]
            axes[-1].set_title(f"{ax_title}, channel {i}")

        if color_mesh is None:
            color_mesh = ax_3d.pcolormesh(X, Y, Z_values, cmap=cm.coolwarm)
        else:
            color_mesh.set_array(Z_values[:-1, :-1])
        color_mesh.set_clim(vmin=-z_cutoff, vmax=+z_cutoff)

        if resonances_indicators:
            for R, (line, text) in enumerate(resonances_indicators, 1):
                mass = kwargs[f"m{R}"]
                line.set_xdata(mass)
                text.set_x(mass + (x_max - x_min) * 0.008)
        else:
            for R in range(1, n_resonances + 1):
                mass = kwargs[f"m{R}"]
                line = ax_3d.axvline(mass, **mass_line_style)
                text = ax_3d.text(
                    x=mass + (x_max - x_min) * 0.008,
                    y=0.95 * y_min,
                    s=f"$m_{R}$",
                    c="red",
                )
                resonances_indicators.append((line, text))

        if kmatrix_type is RelativisticKMatrix:
            x_offset = (x_max - x_min) * 0.015
            if threshold_indicators:
                for i, (line_thr, line_diff, text_thr, text_diff) in enumerate(
                    threshold_indicators
                ):
                    m_a = kwargs[f"m_a{i}"]
                    m_b = kwargs[f"m_b{i}"]
                    s_thr = m_a + m_b
                    m_diff = abs(m_a - m_b)
                    line_thr.set_xdata(s_thr)
                    line_diff.set_xdata(m_diff)
                    text_thr.set_x(s_thr)
                    text_diff.set_x(m_diff - x_offset)
            else:
                colors = cm.plasma(np.linspace(0, 1, n_channels))
                for i, color in enumerate(colors):
                    m_a = kwargs[f"m_a{i}"]
                    m_b = kwargs[f"m_b{i}"]
                    s_thr = m_a + m_b
                    m_diff = abs(m_a - m_b)
                    line_thr = ax.axvline(s_thr, c=color, linestyle="dotted")
                    line_diff = ax.axvline(m_diff, c=color, linestyle="dashed")
                    text_thr = ax.text(
                        x=s_thr,
                        y=0.95 * y_min,
                        s=f"$m_{{a{i}}}+m_{{b{i}}}$",
                        c=color,
                        rotation=-90,
                    )
                    text_diff = ax.text(
                        x=m_diff - x_offset,
                        y=0.95 * y_min,
                        s=f"$m_{{a{i}}}-m_{{b{i}}}$",
                        c=color,
                        rotation=+90,
                    )
                    threshold_indicators.append(
                        (line_thr, line_diff, text_thr, text_diff)
                    )
            for i, (_, line_diff, _, text_diff) in enumerate(
                threshold_indicators
            ):
                m_a = kwargs[f"m_a{i}"]
                m_b = kwargs[f"m_b{i}"]
                s_thr = m_a + m_b
                m_diff = abs(m_a - m_b)
                if m_diff > x_offset + 0.01 and s_thr - abs(m_diff) > x_offset:
                    line_diff.set_alpha(0.5)
                    text_diff.set_alpha(0.5)
                else:
                    line_diff.set_alpha(0)
                    text_diff.set_alpha(0)

    # Create switch for imag/real/abs
    name = "complex_rendering"
    sliders._sliders[name] = ipywidgets.RadioButtons(
        options=["imag", "real", "abs"],
        description=R"\(s\)-plane plot",
    )
    sliders._arg_to_symbol[name] = name

    # Create cut-off slider for z-direction
    name = "z_cutoff"
    sliders._sliders[name] = ipywidgets.FloatSlider(
        value=30,
        min=+1,
        max=+100,
        step=1,
        description=R"\(z\)-cutoff",
    )
    if kmatrix_type in {NonRelativisticPVector, NonRelativisticKMatrix}:
        sliders._sliders[name].value = 1
        sliders._sliders[name].min = 0.01
        sliders._sliders[name].max = 2
        sliders._sliders[name].step = 0.01
    sliders._arg_to_symbol[name] = name

    # Link sliders
    if kmatrix_type is RelativisticKMatrix:
        for i in range(n_channels):
            ipywidgets.dlink(
                (sliders[f"m_a{i}"], "value"),
                (sliders[f"m_b{i}"], "value"),
            )

    # Create GUI
    sliders_copy = dict(sliders)
    h_boxes = []
    for R in range(1, n_resonances + 1):
        buttons = [sliders_copy.pop(f"m{R}")]
        if n_channels == 1:
            if kmatrix_type is RelativisticKMatrix:
                buttons += [
                    sliders_copy.pop(
                        sliders.symbol_to_arg[fR"\Gamma_{{{R},0}}"]
                    ),
                    sliders_copy.pop(
                        sliders.symbol_to_arg[fR"\gamma_{{{R},0}}"]
                    ),
                ]
            if kmatrix_type in {
                NonRelativisticPVector,
                NonRelativisticKMatrix,
            }:
                buttons += [
                    sliders_copy.pop(sliders.symbol_to_arg[f"Gamma{R}"]),
                    sliders_copy.pop(
                        sliders.symbol_to_arg[fR"\gamma_{{{R},0}}"]
                    ),
                ]
        h_box = ipywidgets.HBox(buttons)
        h_boxes.append(h_box)
    remaining_sliders = sorted(
        sliders_copy.values(), key=lambda s: (str(type(s)), s.description)
    )
    if n_channels == 1:
        remaining_sliders.remove(sliders["i"])
    ui = ipywidgets.VBox(h_boxes + remaining_sliders)
    output = ipywidgets.interactive_output(plot3, controls=sliders)
    display(ui, output)

{{ run_interactive }}

In [ ]:
%%time
plot(
    RelativisticKMatrix,
    n_resonances=2,
    n_channels=1,
    angular_momentum=L,
    phsp_factor=dynamics.PhaseSpaceFactor,
)

In [ ]:
if STATIC_WEB_PAGE:
    output_path = "k-matrix.png"
    plt.savefig(output_path, dpi=150)
    display(Image(output_path))